In [1]:
from keras.optimizers import RMSprop #RMSprop is a gradient based optimization technique used 
#in training neural networks. 

from keras.preprocessing.image import ImageDataGenerator #Image data augmentation is a 
#technique that can be used to artificially expand the size of a training dataset by creating 
#modified versions of images in the dataset. ... The Keras deep learning neural network library
#provides the capability to fit models using image data augmentation via the ImageDataGeneratorclass.

import cv2 #OpenCV-Python is a library of Python bindings designed to solve computer vision problems. 

from keras.models import Sequential #The sequential API allows you to create models 
#layer-by-layer for most problems. ... Alternatively, the functional API allows you to create 
#models that have a lot more flexibility as you can easily define models where layers connect to more than just the previous and next layers.

from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout

from keras.models import Model, load_model

from keras.callbacks import TensorBoard, ModelCheckpoint # TensorBoard is a tool for providing
#the measurements and visualizations needed during the machine learning workflow. 
#It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, 
#projecting embeddings to a lower dimensional space, and much more.. The ModelCheckpoint 
#callback class allows you to define where to checkpoint the model weights, how the file 
#should named and under what circumstances to make a checkpoint of the model.

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score#The F-score, also called the F1-score, is a measure of a 
#model's accuracy on a dataset. ... The F-score is a way of combining the precision and recall
#of the model, and it is defined as the harmonic mean of the model's precision and recall.

from sklearn.utils import shuffle #The shuffle is used to shuffle your matrices randomly. 
#Programmatically, random sequences are generated using a seed number.

import imutils #Imutils are a series of convenience functions to make basic image processing 
#functions such as translation, rotation, resizing, skeletonization, and displaying Matplotlib
#images easier with OpenCV and both Python 2.7 and Python 3

import numpy as np


ModuleNotFoundError: No module named 'keras'

In [3]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
# 100 filters (3,3) and then we use Max Pooling to reduce the spatial dimensions of the output volume.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dropout (Dropout)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 50)                6

In [5]:
TRAINING_DIR = "./train"
#Generate batches of tensor image data with real-time data augmentation.

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
#The data will be looped over (in batches).
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [6]:
# to save the Keras model or model weights at some frequency.
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath ,monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [8]:
#A callback is an object that can perform actions at various stages of training 
#(e.g. at the start or end of an epoch, before or after a single batch, etc).
#fit_generator() function first accepts a batch of the dataset, then performs backpropagation 
#on it, and then updates the weights in our model. For the number of epochs specified
#(10 in our case) the process is repeated.

history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
 44/132 [=========>....................] - ETA: 42s - loss: 0.6964 - acc: 0.5227

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model(filepath)# the '.h5' suffix causes weights to be saved in HDF5 format.
results={0:'Without Mask',1:'Mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
#To capture a video, you need to create a VideoCapture object. Its argument can be either 
#the device index or the name of a video file. Device index is just the number to specify 
#which camera. Normally one camera will be connected (as in my case). So I simply pass 0 (or -1)
#. You can select the second camera by passing 1 and so on. After that, you can capture frame-by-frame. 
#But at the end, don’t forget to release the capture.
cap = cv2.VideoCapture(0) 
#OpenCV already contains many pre-trained classifiers for face, eyes, smile etc. 
#Those XML files are stored in opencv/data/haarcascades/ folder
haarcascade = cv2.CascadeClassifier('/usr/local/lib/python3.8/site-packages/cv2/data/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE VIDEO',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()
    

In [ ]:
print(im)

In [ ]:
im.shape